![Shark attacks, a project by Roberto Henríquez Perozo. Data Analytics Bootcamp at IronHack](shark-attacks.png)

<center>You can clone the repo of this project at <br>https://github.com/rihp/shark-attacks </center>

 <center><h1> PART II <br> Data analysis and hypothesis formulation</h1> </center>
 
 

In [ ]:
# You should have an exported.csv file. 
# We generated it while following along the 'PART I' of this project. 
# Do you have it?
!ls
# If you don't, you can download the 'PART I' from: https://github.com/rihp/shark-attacks

In [ ]:
# Importing modules
import pandas as pd
import random
import matplotlib

%matplotlib inline

In [ ]:
# Loading the cleaned dataset as a dataframe
df = pd.read_csv('exported.csv')
df

In [ ]:
# Review the columns of the exported dataset
df.columns

# 🎣️  BASIC GRAPHS

In [ ]:
# Total Fatalities
s_fatal = df.Fatal.value_counts()
s_fatal.plot.pie(title='Distribution of total fatalities')

In [ ]:
# Types of attack
s_types = df.Type.value_counts()
s_types.plot.pie(title='Distribution of all Types of attacks')

In [ ]:
# This plot is a mess, don't graph it
df.Location[df.Location.value_counts() > 20].value_counts().plot.pie()

In [ ]:
# IDEA: make bins to filter the years when it happened

In [ ]:
Years2p = list(range(1800,2020,2))
df.Year[df.Year.isin(Years2p)].value_counts().plot.bar()

In [ ]:
df["Decade"] = pd.cut(df.Year,
                            bins=[1800,1850,1900,1950,2000,2050])
                            #, 
#                            labels=range(1800,2020,10))
df.Decade.value_counts().plot.density()

In [ ]:
df.Species2[~df.Species2.isin(['INVALID ENTRY',
                               'OTHER / NOT KNOWN'])
           ].value_counts().plot.bar()

In [ ]:
df[~df['Fatal'].isin(['UNKNOWN'])][['Species2', 'Fatal']]

# I want to make a new column that specifies, the species and if it was deadly or not
Firts, I'll take any values which don't contribute to our research, and focus only on the deadliest species

In [ ]:
fatals_filetred = ~df.Fatal.isin(['UNKNOWN'])

In [ ]:
df.Species2.value_counts()

In [ ]:
# Define the sharks I want to consider
killer_sharks =['White shark',
                'Tiger shark',
                'Bull shark',
                'Nurse shark', 
                'Reef shark',
                'Bronze whaler shark']

# How many samples of each species are there?
print(df.Species2[df.Species2.isin(killer_sharks)].value_counts())

# Create a dataframe containing only the indexes with those species
species_fatality = df[df.Species2.isin(killer_sharks)]
#df.Species2.value_counts()

In [ ]:
species_fatality = df[df.Species2.isin(killer_sharks)]

In [ ]:
species_fatality = species_fatality#.reset_index()
species_fatality[['Fatal', 'Species2']]
species_fatality[fatals_filetred]#.value_counts()

In [ ]:
dfx = species_fatality[fatals_filetred]

In [ ]:
dfx

In [ ]:
print(dfx.Fatal.value_counts())
print()
print(dfx.Species2.value_counts())

# 🦈️At this point, the data is clean enough to create the plot
## Grouping by Species, and then Fatality

In [ ]:
import numpy as np

In [ ]:
dfx['Victim_died'] = np.where(dfx.Fatal.isin(['Y']), True, False)
dfx['Victim_survived'] = np.where(dfx.Fatal.isin(['N']), True, False)

In [ ]:
toplot =dfx.groupby(['Species2','Fatal']).agg({'Victim_survived':'sum', 'Victim_died':'sum'})

In [ ]:
toplot.plot.bar()

In [ ]:
toplot

In [ ]:
toplot.T.columns

In [ ]:
examine = 'White shark'
display(toplot.T[f'{examine}'].plot.bar(title=f'{examine} mortality'))

In [ ]:
examine = 'Tiger shark'
display(toplot.T[f'{examine}'].plot.bar(title=f'{examine} mortality'))

In [ ]:
examine = 'Bull shark'
toplot.T[f'{examine}'].plot.bar(title=f'{examine} mortality')

In [ ]:
examine = 'Nurse shark'
toplot.T[f'{examine}'].plot.bar(title=f'{examine} mortality')

In [ ]:
dfx[['Species2','Fatal2']].groupby('Species2').agg({'Fatal':"count"})

In [ ]:
sx = df.groupby(["Fatal", "Species2"]).agg({"Date":["min", "max"]})
sx

In [ ]:
sx = df.groupby(["Fatal", "Species2"]).agg({"Date":["min", "max"]})
sx

In [ ]:
def filt(x):
    if x =='UNKNOWN':
        return None
    else:
        return x
df.reset_index()
Fatal_na = df.Fatal.isin(['UNKNOWN'])
Species_na = df.Species2.isin(['INVALID ENTRY', 'OTHER / NOT KNOWN'])
sx = df.loc[~Fatal_na].groupby(["Fatal", "Species2"]).agg({"Type":"count"})
sy = df.loc[~Species_na]
# avoid.value_counts()
#Species_na.value_counts()
#df[~Fatal_na, ~Species_na]
sp_na = df[~Species_na]
df_nonull = sp_na[~Fatal_na]

In [ ]:
df_ok = df_nonull.reset_index()
# df_ok.head().plot.scatter(df_ok.Species2,df_ok.Fatal)

In [ ]:
matplotlib.pyplot.scatter(df_nonull.Fatal, df_nonull.Species2)


In [ ]:
dfx = df[['Species2','Fatal']]
dfx.isnull().sum()

In [ ]:
avoid1 = ~df['Fatal'].isin(['UNKNOWN'])
avoid2 = ~df['Species2'].isin(['OTHER / NOT KNOWN', 'INVALID ENTRY'])
df2p = df[avoid1][avoid2][['Species2', 'Fatal']]
#df2p.plot.bar('Fatal', 'Species2')

In [ ]:
df2p.Fatal.values

In [ ]:
df2p.Species2.values

In [ ]:
Yes = (lambda x: True if x == 'Y' else False)
df2p.groupby('Species2').agg({"Fatal" : Yes})

In [ ]:
df.columns